## Descripción
Predicción de síntomas en un paciente con una cepa específica del virus Zika usando datos clínicos específicos y métodos de Machine Learning.
?? Sólo tenemos un output. Yo digo que es si muere o no.
## Métodos
Muchos
**Procesado de datos**
- Buscar errores o missing values y eliminar estas filas.
    No había ninguno.
- Cambiar escala de features.
    Ya están estandarizadas.
    
- Feature Selection.
    Probamos pero no lo hacemos por ser características médicas.
    
- Balanceo de clases.
    Hacemos undersampling para balancear.
- Separación en conjuntos de entrenamiento, validación y test.

**Machine Learning**

- Use any method of ML / Deep Learning you consider to obtain the best model for your task/problem.
- Use metrics of ML to decide the best model.
- Save the model if possible as file.

## Resultados y discusión

In [1]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler    
from sklearn.metrics import roc_curve, auc, confusion_matrix
from sklearn.utils import shuffle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import (train_test_split, GridSearchCV,
                                     ParameterGrid)

In [2]:
# Cargamos dataset
df = pd.read_csv(os.path.join('ds_Zika.csv'))

In [3]:
# Comprobamos si falta algún valor
attr = []
# Iteramos los atributos del dataset
for i in df.columns:
    # Comprobamos los valores de cada atributo
    attr.append(np.all(df[i].notna()))
# Ningún atributo tiene missing values:
np.all(np.array(attr))

True

In [4]:
# Balanceo de clases
# Comprobamos la proporción
np.mean(df["Output"])

0.18557084417999256

In [5]:
# Undersampling
# Número de casos positivos
nposit = len(df[df['Output'] == 1])
# Índices de casos negativos
inegat = df[df.Output == 0].index
# Cogemos aleatoriamente un número de índices negativos igual al número de casos positivos
random_indices = np.random.choice(inegat, nposit, replace=False)
# Obtenemos todos los índices positivos
iposit = df[df.Output == 1].index
# Unimos las dos listas de índices
new_indices = np.concatenate([iposit, random_indices])
df = df.loc[new_indices]
# y las desordenamos
df = shuffle(df).reset_index(drop=True)

X = df.drop('Output', axis=1)
y = df['Output']

Xdata = X.values
ydata = y.values

In [6]:
# Separación en conjuntos de entrenamiento y test:
#ssplit = ShuffleSplit(n_splits=100, test_size=0.2)
#for i_train, i_test in ssplit.split(Xdata):

In [7]:
# Suprimimos warnings para legibilidad (nota: no recomendamos hacer esto)
import warnings
warnings.filterwarnings("ignore")

In [8]:
# # K Neighbors
# knb = KNeighborsClassifier()

# # LDC:
# ldc = LinearDiscriminantAnalysis()

# # Bayesian:
# gnb = GaussianNB()

# # Multilayer Perceptron
# mlp = MLPClassifier()

# # SVM:
# svm = SVC()

# # Random Forest
# rndf = RandomForestClassifier()

# models = [knb, ldc, gnb, mlp, svm, rndf]
# model_names = ['KNeighborsClassifier', 'LinearDiscriminantAnalysis', 'GaussianNB', 'MLPClassifier', 'SVC', 'RandomForestClassifier']
# scores = []
# for i,model in enumerate(models):
#     score = cross_val_score(model, Xdata, ydata, cv=10)
#     print(model_names[i], ' - Mean:', np.mean(score), ' - Standard Deviation:', np.std(score))
# warnings.filterwarnings("default")

In [12]:
X_train, X_test, y_train, y_test = train_test_split(Xdata, ydata,
                                                    test_size=0.25,
                                                    stratify=ydata)

param_grid = {'n_estimators': range(1, 101, 10),
               'max_depth': range(1, 26, 5),
               'min_samples_leaf': range(1, 11, 5)}

alg = RandomForestClassifier(random_state=1234)
grid = GridSearchCV(alg, param_grid=param_grid, cv=10)
grid.fit(X_train, y_train)
print(grid.best_params_)
print('validation accuracy:', grid.score(X_train, y_train))
print('test accuracy:', grid.score(X_test, y_test))


{'max_depth': 16, 'min_samples_leaf': 1, 'n_estimators': 91}
validation accuracy: 0.9993319973279893
test accuracy: 0.9398797595190381


In [13]:
c = pd.DataFrame(grid.cv_results_)
c['train_test_difference'] = c.mean_train_score - c.mean_test_score
max_diff = 0.02
min_test_score = 0.90
c_sorted = c[(c.mean_test_score >= min_test_score) & (c.train_test_difference <= max_diff)][(
    ['mean_train_score', 'mean_test_score','train_test_difference'])].sort_values('mean_test_score', ascending=False)
print(c_sorted)
i = c_sorted.first_valid_index()
max_depth = c.param_max_depth.iloc[i]
min_samples_leaf = c.param_min_samples_leaf.iloc[i]
n_estimators = c.param_n_estimators.iloc[i]
print('Max depth: {} - Min. samples leaf: {} - Num. estimators: {}'.format(max_depth, min_samples_leaf, n_estimators))

    mean_train_score  mean_test_score  train_test_difference
79          0.945075         0.925184               0.019892
58          0.942552         0.924516               0.018036
59          0.941810         0.923848               0.017963
57          0.941513         0.922512               0.019001
56          0.941958         0.922512               0.019447
55          0.942033         0.922512               0.019521
53          0.940919         0.921176               0.019743
28          0.933793         0.917836               0.015958
22          0.934980         0.915164               0.019817
29          0.933942         0.915164               0.018778
26          0.932680         0.915164               0.017516
24          0.933867         0.914496               0.019372
27          0.932828         0.914496               0.018333
25          0.933125         0.914496               0.018630
33          0.925036         0.912492               0.012544
36          0.923476    

In [20]:
best = RandomForestClassifier(random_state=1234)
best.fit(X_train, y_train)
print('validation accuracy:', best.score(X_train, y_train))
print('test accuracy:', best.score(X_test, y_test))

validation accuracy: 0.9953239812959251
test accuracy: 0.9138276553106213
